In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import Image
from typing import List, Dict
import copy
import json
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tempfile, os

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

supported_models = ['resnet18', 'resnet50', 'resnet101', 'vgg16_bn', 'vgg19_bn', 'inception_v3']
model_name = supported_models[2]
verify_model_name = supported_models[4]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# if str(device) == 'cpu':
#     raise RuntimeError("cuda is NOT available!!")

benign_pic = './data/cat/Cat04.jpg'
benign_pic = './data/duck/Duck02.jpg'
benign_pic = './data/mouse/Mouse06.jpg'


target_id = 508 # computer_keyboard
target_id = 99 # goose

# Constants

In [ ]:
imagenet_mean=torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
imagenet_std=torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

In [ ]:
# epsilon = 8 / 255. / imagenet_std
epsilon = 8 / 255. * imagenet_std
epsilon

# Utilities

## Visualize

In [ ]:
# visualize(preprocess_image(benign_pic), adv_x, predicted_class, benign_confidence, adv_predicted_class, adv_confidence)
def visualize(x, adv, benign_label:int, adv_label:int, benign_confidence:float, adv_confidence:float, height:int=10, width:int=20):
    def restore(x):
        # x = x * imagenet_std + imagenet_mean
        return x

    x, adv = restore(x.detach().cpu()), restore(adv.detach().cpu())
    x, adv = x.numpy().transpose([0, 2, 3, 1]), adv.numpy().transpose([0, 2, 3, 1]) # transpose (bs, C, H, W) back to (bs, H, W, C)
    plt.figure(figsize=(height, width))
    
    plt.subplot(1, 2, 1)
    # predicted_class, predicted_classname, confidence = classify_image(model, benign_pic, class_labels)
    plt.title(f"Benign: {class_labels[benign_label]} (confidence: {benign_confidence:.1%})")
    plt.axis('off')
    plt.imshow(x.squeeze())
    
    plt.subplot(1, 2, 2)
    plt.title(f"Adversary: {class_labels[adv_label]} (confidence: {adv_confidence:.1%})")
    plt.axis('off')
    plt.imshow(adv.squeeze())

    plt.tight_layout()

## Store & Load

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import re

def store_img_from_tensor(tensor_image, img_path:str):
    if not re.match(r'.*\.png$', img_path, re.IGNORECASE):
        raise TypeError(f"We have to store image file to png due to the `loss nature of JPEG format`!")

    # tensor_image = tensor_image * imagenet_std + imagenet_mean
    # Define the transformation to apply to the tensor
    transform = transforms.ToPILImage()
    # Apply the transformation to the tensor
    pil_image = transform(tensor_image.squeeze())
    # Save the PIL image to disk
    pil_image.save(img_path) # , quality=100) - not necessary as png, unlike jpeg, will not lose quality
    
def load_img_to_tensor(img_path:str):
    # import ipdb; ipdb.set_trace()
    # Load the image using PIL
    pil_image = Image.open(img_path).convert('RGB')
    
    # Define the transformation to apply to the image
    img_loader = transforms.Compose([
        transforms.ToTensor(), # Convert the image to a tensor
        # transforms.Resize(size=8), # 256),
        # transforms.CenterCrop(size=6), #224),
        # transforms.Normalize(mean=imagenet_mean, std=imagenet_std) # Normalize the image
    ])

    # Apply the transformation to the PIL image
    tensor_image = img_loader(pil_image)
    tensor_image = torch.unsqueeze(tensor_image, 0)  # Add a batch dimension

    return tensor_image

# Create model

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py).

Model label list is ImageNet labels which can be found [here](https://files.fast.ai/models/imagenet_class_index.json)

In [ ]:
class ModelFactory():
    _instance = None
    _supported_models = []
    _models = {}
    _class_labels = None

    def _fill_classlabels(self):
        class_file = 'imagenet_class_index.json'
        with open(class_file, 'r') as f:
            f_contents = f.read()
        class_labels = json.loads(f_contents)
        self._class_labels = {int(k):v[1] for k, v in class_labels.items()}
        
    def __new__(self, supported_models, *args, **kwargs):
        if not self._instance:
            self._instance = super(ModelFactory, self).__new__(self, *args, **kwargs)
            self._supported_models = copy.deepcopy(supported_models)
            self._models = dict()
            self._fill_classlabels(self)
        return self._instance
            
    def get_supported_models(self)->List[str]:
        return self.supported_models

    def get_model(self, model_name):
        m = model_name.lower()
        # import ipdb; ipdb.set_trace()
        try:
            self._supported_models[self._supported_models.index(m)]
        except ValueError as ve:
            raise ValueError(f"Not supported model: {model_name} - {ve.args}")
        model = self._models.get(m)
        if not model: # model is not yet initialized
            print(f"{m} is not yet initialized, create a new one!")
            model = models.get_model_builder(m)(pretrained=True).to(device)
            model.requires_grad_(False)
            model.eval()
            self._models[m] = model
        else:
            print(f"{m} is already initialized, return directly!")
        return self._models.get(m)

    def get_class_labels(self):
        return self._class_labels

model_factory = ModelFactory(supported_models)
class_labels = model_factory.get_class_labels()

# Classify picture

In [ ]:
# Preprocess the image
def preprocess_image(image_path:str):
    preprocessed_image = load_img_to_tensor(image_path)

    return preprocessed_image.to(device)

def get_logits(model, image_path:str):
    preprocessed_image = preprocess_image(image_path)
    with torch.no_grad():
        logits = model(preprocessed_image)
    return logits

# Classify the image
def classify_image(model, image_path:str, class_labels:Dict[int, str]):
    logits = get_logits(model, image_path)
    # import ipdb; ipdb.set_trace()
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities.squeeze()).item()
    predicted_classname = class_labels[predicted_class]
    confidence = probabilities.squeeze()[predicted_class]
    return predicted_class, predicted_classname, confidence, probabilities

# Classify via tensor
def classify_tensor(model, x, class_labels:Dict[int, str]):
    with torch.no_grad():
        logits = model(x)
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities.squeeze()).item()
    predicted_classname = class_labels[predicted_class]
    confidence = probabilities.squeeze()[predicted_class]
    return predicted_class, predicted_classname, confidence, probabilities
    

## Check result of benign examples

In [ ]:
test_tensor_image = load_img_to_tensor(benign_pic).to(device)
print(f"test_tensor_image.shape: {test_tensor_image.shape}")

test_model_name = supported_models[0]
test_model = model_factory.get_model(test_model_name)

y_hat = F.softmax(test_model(test_tensor_image))

test_pred = torch.argmax(y_hat).detach().cpu().item()
test_pred_name = class_labels[test_pred]
test_confidence = y_hat[0][test_pred]
print(f"Predicted name: {test_pred_name} with index: {test_pred:d}. Confidence: {test_confidence:.2%}")

In [ ]:
model = model_factory.get_model(model_name)
predicted_class, predicted_classname, confidence, probabilities = classify_image(model, benign_pic, class_labels)
print(f'Predicted class: {predicted_classname} (No: {predicted_class:d}, with confidence: {confidence:.1%})')

# gradient

In order to calculate gradient, we'll use cross-entropy loss here:

$$\text{CrossEntropyLoss} = - \sum{_{i=1}^{N} \left( y_{i}\log{(p_{i})}+(1-y_{i})\log{(1-p_{i})} \right)}$$

Note, the `F.cross_entropy` or `nn.CrossEntropyLoss` is calculated as follows:

1. Apply a softmax function to the raw scores to get a probability distribution.
2. Compute the negative log-likelihood loss against the given labels.

So, the result of the following code is NOT `tensor(0.)` but `tensor(0.9048)` because the `F.cross_entropy` calculates `softmax` first on the input (`pred`) and thus the value of the zeroth index is no longer `1.`:

```python
loss_fn = F.cross_entropy
pred = torch.tensor([[1., 0, 0, 0, 0]])
labels = torch.tensor([0])
loss_fn(pred, labels)
```

Comparing with the following code which outputs resulting loss very close to `0`:

```python
loss_fn = F.cross_entropy
pred = torch.tensor([[10., 0, 0, 0, 0]])
labels = torch.tensor([0])
loss_fn(pred, labels)
```

Here, because the zeroth-index of `pred` is large enough comparing to others value, after `softmax`, the zeroth-index of the output (the input of the `negative log-likelihood`) is very close to `1.`, the output loss is thus very close the `0.`.

## non-target gradient

In [ ]:
def non_target_grad(model, pic:str, ground_truth:int, loss_fn=F.cross_entropy):
    y = torch.tensor([ground_truth]).to(device)
    x = preprocess_image(pic)
    # import ipdb; ipdb.set_trace()
    x.requires_grad = True
    y_hat = model(x)
    loss = -loss_fn(y_hat, y)
    print(f"loss = {loss:.4f}")
    loss.backward()
    return x.grad

model = model_factory.get_model(model_name)
x_grad = non_target_grad(model, pic=benign_pic, ground_truth=predicted_class)

print(f"\nx_grad.shape = {x_grad.shape}")

## target gradient

In [ ]:
def target_grad_by_x(model, x, target:int, loss_fn=F.cross_entropy):
    y_target = torch.tensor([target]).to(device)
    
    x0 = x.detach().to('cpu').clone().to(device) # * imagenet_std + imagenet_mean
    x0.requires_grad = True
    y_hat = model(x0)
    
    target_loss = loss_fn(y_hat, y_target)
    adv_loss = torch.tensor([0.]).to(device)
    other_losses = []
    for y_other in class_labels.keys():
        if y_other == target:
            continue
        else:
            other_loss = loss_fn(y_hat, torch.tensor([y_other]).to(device))
            adv_loss += target_loss / other_loss
            other_loss = other_loss.detach().cpu()
            other_losses.append(other_loss)

    adv_loss.backward()
    return x0.grad.cpu(), other_losses, target_loss.cpu(), adv_loss.cpu()

def target_grad_by_file(model, pic:str, target:int, loss_fn=F.cross_entropy):
    x = preprocess_image(pic)
    
    x_grad, other_losses, target_loss, adv_loss = target_grad_by_x(model, x, target, loss_fn)
    return x_grad, other_losses, target_loss, adv_loss

model = model_factory.get_model(model_name)

# x_grad, other_losses, target_loss, adv_loss = target_grad_by_file(model, pic=benign_pic, target=target_id)

# print(f"\nx_grad.shape = {x_grad.shape}")

# Attacking algorithms

## fgsm

In [ ]:
def fgsm(benign_pic:str, model, predicted_class:int, target_id:int):
    x = preprocess_image(benign_pic)
    x_grad, other_losses0, target_loss0, adv_loss0 = target_grad_by_file(model, benign_pic, 
                                                           ground_truth=predicted_class, 
                                                           target=target_id)
    adv_x = x - epsilon * x_grad.sign()
    adv_file = tempfile.NamedTemporaryFile().name + ".png"
    store_img_from_tensor(adv_x, adv_file)
    print(f"{'#' * 20}")
    target_grad_by_file(model, adv_file, ground_truth=predicted_class, target=target_id) # See if the loss decreased
    return adv_x, x_grad, adv_file

# predicted_class, predicted_classname, confidence, probabilities = classify_image(model, benign_pic, class_labels)
# print(f'Predicted class: {predicted_classname} (No: {predicted_class:d}, with confidence: {confidence:.1%})')

# target_id = 508 # computer_keyboard
# adv_x, x_grad, adv_file = fgsm(benign_pic, model, predicted_class, target_id)

# adv_predicted_class, adv_predicted_classname, adv_confidence, adv_probabilities = classify_image(model, adv_file, class_labels)
# print(f'Predicted class: {adv_predicted_classname} (No: {adv_predicted_class:d}, with confidence: {adv_confidence:.1%})')

# visualize(preprocess_image(benign_pic), adv_x, predicted_class, adv_predicted_class)

# os.remove(adv_file)

## ifgsm

In [ ]:
def clamp_tensor_image(tensor_image):
    scale = 255
    tensor_image = tensor_image * scale  # Scale up
    tensor_image = torch.round(tensor_image)  # Round
    tensor_image = torch.clamp(tensor_image, 0, scale) # each item in tensor_image should be inside [0, 255]
    tensor_image = tensor_image / scale  # Scale down
    return tensor_image

In [ ]:
# %%time
# size = [3, 256, 256] # (3, 4, 5)
# aaa0 = torch.rand(size=size)
# for scale in range(200, 10000):
#     aaa = clamp_tensor_image(aaa0, scale=scale)
#     store_img_from_tensor(aaa, '/tmp/aaa.png')
#     aaa1 = load_img_to_tensor('/tmp/aaa.png').squeeze()
#     if aaa.allclose(aaa1) and aaa.allclose(aaa0, atol=1e-2):
#         print(f"scale = {scale}")
#         print("aaa0:\n", aaa0[0][0][:20])
#         print("aaa:\n", aaa[0][0][:20])
#         print("aaa1:\n", aaa1[0][0][:20])
#         break

In [ ]:
def ifgsm(benign_pic:str, model, target_id:int, num_iterate:int=200, alpha=None):
    alpha = alpha if alpha!=None else epsilon / num_iterate
    x = preprocess_image(benign_pic).detach().to('cpu')
    clip_ratio = 1
    clip_arrange = [x + clip_ratio * epsilon, x - clip_ratio * epsilon]
    adv_x = x.clone()

    other_losses_list = []
    target_losses = []
    adv_losses = []
    display_interval = 10
    for i in range(num_iterate):
        # adv_file = tempfile.NamedTemporaryFile().name + ".png"
        # store_img_from_tensor(adv_x, adv_file)
        x_grad, other_losses, target_loss, adv_loss = target_grad_by_x(
            model, x=adv_x, 
            target=target_id)
        
        total_other_losses = torch.sum(torch.tensor(other_losses)).item()
        
        if i % display_interval == 0:
            # import ipdb; ipdb.set_trace()
            print(f"total_other_losses = {total_other_losses:.4f},\ttarget_loss = {target_loss:.4f},\tadv_loss = {adv_loss}")
            print(f"quilt's loss = {other_losses[750]:.4f}")
        
        other_losses_list.append(total_other_losses)
        target_losses.append(target_loss)
        adv_losses.append(adv_loss)

        adv_x = adv_x - alpha * x_grad.detach().sign()
        adv_x = torch.max(torch.min(adv_x, clip_arrange[0]), clip_arrange[1]) # clip new adv_x back to [x-epsilon, x+epsilon]
        adv_x = clamp_tensor_image(adv_x)

        # os.remove(adv_file)
    
    return adv_x, other_losses_list, target_losses, adv_losses

# Test

model = model_factory.get_model(model_name)

predicted_class, predicted_classname, confidence, probabilities = classify_image(model, benign_pic, class_labels)
print(f"\nFor {benign_pic}:")
print(f'Predicted class: {predicted_classname} (No: {predicted_class:d}, with confidence: {confidence:.1%}).')
print(f"The target classname is: {class_labels[target_id]} (No. {target_id}) with confidence: {probabilities[0][target_id]:.1%}")

# Check result of adversary examples

## Generate adversary example

In [ ]:
%%time
num_iterate=50
# exclude_list = [predicted_class, 750, 700] # 750: quilt, 700: paper_towel

adv_x, other_losses, target_losses, adv_losses = ifgsm(benign_pic, model, target_id, num_iterate=num_iterate, alpha=35*epsilon/num_iterate)
adv_file = tempfile.NamedTemporaryFile().name + ".png"
store_img_from_tensor(adv_x, adv_file)

## Evaluate using same model

### via tensor

In [ ]:
predicted_class, predicted_classname, confidence, probabilities = classify_image(model, benign_pic, class_labels)
adv_predicted_class, adv_predicted_classname, adv_confidence, adv_probabilities = classify_tensor(model, adv_x.to(device), class_labels)
print(f'Predicted class: {adv_predicted_classname} (No: {adv_predicted_class:d}, with confidence: {adv_confidence:.4%}).')
print(f"The target classname is: {class_labels[target_id]} (No. {target_id}) with confidence: {adv_probabilities[0][target_id]:.4%}")

In [ ]:
visualize(x=preprocess_image(benign_pic), adv=adv_x, benign_label=predicted_class, adv_label=adv_predicted_class, benign_confidence=confidence, adv_confidence=adv_confidence)

### via stored picture

In [ ]:
store_img_from_tensor(adv_x, adv_file)
adv_x1 = load_img_to_tensor(adv_file)
adv_x.allclose(adv_x1, rtol=1e-6, atol=1e-2)

In [ ]:
predicted_class, predicted_classname, confidence, probabilities = classify_image(model, benign_pic, class_labels)
adv_predicted_class, adv_predicted_classname, adv_confidence, adv_probabilities = classify_image(model, adv_file, class_labels)
print(f"\nFor {adv_file}:")
print(f'Predicted class: {adv_predicted_classname} (No: {adv_predicted_class:d}, with confidence: {adv_confidence:.4%}).')
print(f"The target classname is: {class_labels[target_id]} (No. {target_id}) with confidence: {adv_probabilities[0][target_id]:.4%}")
visualize(x=preprocess_image(benign_pic), adv=adv_x, benign_label=predicted_class, adv_label=adv_predicted_class, benign_confidence=confidence, adv_confidence=adv_confidence)

In [ ]:
shutil.copyfile(adv_file, f"{class_labels[adv_predicted_class]}.png")

## Evaluate using defferent model

### Get a different model

In [ ]:
verify_model = model_factory.get_model(verify_model_name)

In [ ]:
predicted_class, predicted_classname, confidence, probabilities = classify_tensor(model, preprocess_image(benign_pic), class_labels)
predicted_class, predicted_classname, confidence.item()*100

In [ ]:
predicted_class, predicted_classname, confidence, probabilities = classify_tensor(verify_model, adv_x.to(device), class_labels)
predicted_class, predicted_classname, confidence.item()*100

### via tensor

In [ ]:
predicted_class, predicted_classname, confidence, probabilities = classify_image(verify_model, benign_pic, class_labels)
# predicted_class, predicted_classname, confidence, probabilities = classify_tensor(verify_model, preprocess_image(benign_pic), class_labels)
adv_predicted_class, adv_predicted_classname, adv_confidence, adv_probabilities = classify_tensor(verify_model, adv_x.to(device), class_labels)
print(f'Predicted class: {adv_predicted_classname} (No: {adv_predicted_class:d}, with confidence: {adv_confidence:.4%}).')
print(f"The target classname is: {class_labels[target_id]} (No. {target_id}) with confidence: {adv_probabilities[0][target_id]:.4%}")

In [ ]:
visualize(x=preprocess_image(benign_pic), adv=adv_x, benign_label=predicted_class, adv_label=adv_predicted_class, benign_confidence=confidence, adv_confidence=adv_confidence)

### via stored picture

In [ ]:
store_img_from_tensor(adv_x, adv_file)
adv_x1 = load_img_to_tensor(adv_file)
adv_x.allclose(adv_x1, rtol=1e-8, atol=10e-1)

In [ ]:
predicted_class, predicted_classname, confidence, probabilities = classify_image(verify_model, benign_pic, class_labels)
adv_predicted_class, adv_predicted_classname, adv_confidence, adv_probabilities = classify_image(verify_model, adv_file, class_labels)
print(f"\nFor {adv_file}:")
print(f'Predicted class: {adv_predicted_classname} (No: {adv_predicted_class:d}, with confidence: {adv_confidence:.4%}).')
print(f"The target classname is: {class_labels[target_id]} (No. {target_id}) with confidence: {adv_probabilities[0][target_id]:.4%}")
visualize(x=preprocess_image(benign_pic), adv=adv_x, benign_label=predicted_class, adv_label=adv_predicted_class, benign_confidence=confidence, adv_confidence=adv_confidence)

In [ ]:
os.remove(adv_file)